In [ ]:
import pandas as pd
import requests



In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd

# Ruta al archivo recetas.csv en Google Drive
file_path = "/content/drive/MyDrive/recetas.csv"

# Cargar el archivo CSV en un DataFrame
recipes_df = pd.read_csv(file_path, delimiter='|')


# Visualizar las primeras filas del DataFrame para verificar la carga correcta
print(recipes_df.head())


      Id                      Categoria  \
0  74138  Recetas de Aperitivos y tapas   
1  74133  Recetas de Aperitivos y tapas   
2  74154  Recetas de Aperitivos y tapas   
3  74146  Recetas de Aperitivos y tapas   
4  74081  Recetas de Aperitivos y tapas   

                                              Nombre  Valoracion Dificultad  \
0                      Receta de Baleadas hondureñas         NaN       baja   
1                    Receta de Tacos de carne molida         NaN       baja   
2                        Receta de Burritos norteños         NaN   muy baja   
3  Receta de Croquetas de pollo caseras de la abuela         NaN       baja   
4               Receta de Masa para empanadas fritas         NaN       baja   

   Num_comensales Tiempo             Tipo  \
0             2.0    10m              NaN   
1             4.0    30m  Plato principal   
2             5.0    30m         Entrante   
3             4.0    45m         Entrante   
4             3.0    30m         Entrante

In [ ]:
# Eliminar filas con valores nulos
recipes_df.dropna(inplace=True)

# Visualizar la información del DataFrame después del preprocesamiento
print(recipes_df.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6176 entries, 7 to 22365
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  6176 non-null   int64  
 1   Categoria           6176 non-null   object 
 2   Nombre              6176 non-null   object 
 3   Valoracion          6176 non-null   float64
 4   Dificultad          6176 non-null   object 
 5   Num_comensales      6176 non-null   float64
 6   Tiempo              6176 non-null   object 
 7   Tipo                6176 non-null   object 
 8   Link_receta         6176 non-null   object 
 9   Num_comentarios     6176 non-null   int64  
 10  Num_reviews         6176 non-null   int64  
 11  Fecha_modificacion  6176 non-null   object 
 12  Ingredientes        6176 non-null   object 
dtypes: float64(2), int64(3), object(8)
memory usage: 675.5+ KB
None


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Inicializar el vectorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Crear la matriz de características TF-IDF
tfidf_matrix = tfidf_vectorizer.fit_transform(recipes_df['Nombre'])

# Visualizar la forma de la matriz de características TF-IDF
print("Shape of TF-IDF matrix:", tfidf_matrix.shape)


Shape of TF-IDF matrix: (6176, 2245)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Calcular la similitud de coseno entre las recetas
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)


In [ ]:
def get_similar_recipes(recipe_name, cosine_sim=None, df=None):
    if cosine_sim is None or df is None:
        print("Error: Debes proporcionar la matriz de similitud y el DataFrame.")
        return []

    try:
        idx = df[df['Nombre'] == recipe_name].index[0]
    except IndexError:
        print("La receta no se encuentra en el DataFrame.")
        return []

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    recipe_indices = [i[0] for i in sim_scores]
    return df['Nombre'].iloc[recipe_indices]

# Ejemplo de uso de la función de recomendación
similar_recipes = get_similar_recipes("Receta de Salchichas al vino con patatas", cosine_sim=cosine_sim, df=recipes_df)
print(similar_recipes)



15468                 Receta de Mermelada de limón casera
15428             Receta de Mermelada de frambuesa casera
15561               Receta de Mermelada de durazno casera
15450             Receta de Mermelada de melocotón casera
15866               Receta de Mermelada de cerezas casera
18172               Receta de Mermelada de naranja amarga
16737                       Receta de Mermelada de tomate
16828                      Receta de Mermelada de manzana
16040                     Receta de Mermelada de calabaza
18171    Receta de Mermelada de naranja amarga y jengibre
Name: Nombre, dtype: object


In [ ]:
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Montar Google Drive
drive.mount('/content/drive')

# Ruta al archivo CSV en Google Drive
file_path = "/content/drive/MyDrive/recetas.csv"

# Cargar el DataFrame con los datos de las recetas
recipes_df = pd.read_csv(file_path, delimiter='|')

# Obtener las características (matriz TF-IDF) y las etiquetas de las recetas
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(recipes_df['Nombre'])  # Utilizamos los nombres de las recetas como características
labels = recipes_df['Categoria']  # Supongamos que 'Categoria' es la columna que contiene las etiquetas de las recetas

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, labels, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo SVM
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Evaluar el modelo en el conjunto de prueba
y_pred = svm_model.predict(X_test)
print(classification_report(y_test, y_pred))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
                               precision    recall  f1-score   support

           Consejos de cocina       0.94      0.67      0.78        24
           Cócteles y bebidas       0.95      0.97      0.96       153
Recetas de Aperitivos y tapas       0.68      0.72      0.70       408
Recetas de Arroces y cereales       0.79      0.90      0.84       163
       Recetas de Aves y caza       0.90      0.90      0.90       325
             Recetas de Carne       0.84      0.79      0.82       426
         Recetas de Ensaladas       0.95      0.94      0.94       200
  Recetas de Guisos y Potajes       0.70      0.49      0.57        80
  Recetas de Huevos y lácteos       0.80      0.80      0.80       169
         Recetas de Legumbres       0.72      0.80      0.76       101
          Recetas de Mariscos       0.77      0.77      0.77       155
    Recetas de Pan

In [ ]:
from google.colab import drive
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

# Montar Google Drive
drive.mount('/content/drive')

# Ruta al archivo CSV en Google Drive
file_path = "/content/drive/MyDrive/recetas.csv"

# Cargar el DataFrame con los datos de las recetas
recipes_df = pd.read_csv(file_path, delimiter='|')

# Obtener las características (matriz TF-IDF) y las etiquetas de las recetas
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(recipes_df['Nombre'])  # Utilizamos los nombres de las recetas como características
labels = recipes_df['Categoria']  # Supongamos que 'Categoria' es la columna que contiene las etiquetas de las recetas

# Inicializar y entrenar el modelo SVM
svm_model = SVC(kernel='linear')
svm_model.fit(tfidf_matrix, labels)

# Definir función para obtener recomendaciones
def get_recommended_recipes(recipe_name, recipes_df, tfidf_vectorizer, svm_model):
    # Transformar el nombre de la receta utilizando el vectorizador TF-IDF
    recipe_tfidf = tfidf_vectorizer.transform([recipe_name])

    # Predecir la categoría de la receta utilizando el modelo SVM
    predicted_category = svm_model.predict(recipe_tfidf)[0]

    # Filtrar las recetas que pertenecen a la categoría predicha
    recommended_recipes = recipes_df[recipes_df['Categoria'] == predicted_category]['Link_receta'].tolist()

    return recommended_recipes

# Ejemplo de uso
recipe_name = "Receta de salchichas"
recommended_recipes = get_recommended_recipes(recipe_name, recipes_df, tfidf_vectorizer, svm_model)
print("Recetas recomendadas:")
print(recommended_recipes)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Recetas recomendadas:
['https://www.recetasgratis.net/receta-de-entrana-al-horno-74117.html', 'https://www.recetasgratis.net/receta-de-asado-de-tira-74141.html', 'https://www.recetasgratis.net/receta-de-poc-chuc-74157.html', 'https://www.recetasgratis.net/receta-de-salchichas-a-la-cerveza-74149.html', 'https://www.recetasgratis.net/receta-de-chifrijo-74123.html', 'https://www.recetasgratis.net/receta-de-mollejas-de-cordero-en-salsa-74099.html', 'https://www.recetasgratis.net/receta-de-arroz-con-carne-molida-74041.html', 'https://www.recetasgratis.net/receta-de-chinchulines-al-horno-74038.html', 'https://www.recetasgratis.net/receta-de-callos-a-la-madrilena-de-la-abuela-74012.html', 'https://www.recetasgratis.net/receta-de-churrasco-argentino-73998.html', 'https://www.recetasgratis.net/receta-de-estofado-de-carne-con-patatas-73925.html', 'https://www.recetasgr